# Architecture détaillée des réseaux de détection en temps réel #

Dans ce notebook, on va regarder en détail quelle est l'architecture des réseaux YOLO (v1, v2, v3), SSD, et RetinaNet afin de mieux comprendre comment fonctionne la prédiction de bounding boxes dans une image.

## YOLO v1 ##

**Architecture globale** : 24 couches de convolution suivies de 2 couches fully connected (9 couches de convolution pour Fast YOLO au lieu de 24, YOLO a une vitesse de 45 fps tandis que Fast YOLO a une vitesse de 150 fps).

**Architecture détaillée**

| Nom de la couche | Taille du noyau, du filtre ou de la couche | Nombre de filtres de convolution | Stride |
| :------------ | :------------ | :------------ | :------------ |
| Convolution Layer | 7x7 | 64 | 2 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolution Layer | 3x3 | 192 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 1x1 | 512 | 2 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 1x1 | 512 | 2 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 3x3 | 1024 | 2 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Fully Connected Layer | 7x7x1024 | N/A | N/A |
| Fully Connected Layer | 1x4096 | N/A | N/A |
| Sortie | 7x7x30 | N/A | N/A |

Les 20 premières couches convolutives du réseau ont été entrainées sur ImageNet (BD du challenge de classification de 1000 classes). Le pré-traitement se fait avec des images de résolution 224 x 224. On ajoute ensuite 4 couches de convolution suivies de deux couches fully connected qui sont en charge de la détection d'objet. La résolution de la BD est ensuite passée à 448x448 puisque la détection d'objet a besoin d'images de meilleure résolution pour être optimales. La couche finale prédit les bounding boxes et la probabilité d'appartenance à chacune des classes. 

**Prédiction des bounding boxes**

Chaque image analysée est divisée en une grille de taille 7x7, ce qui donne 49 cellules. Pour chacune de ces cellules, 2 bounding boxes sont prédites, ce qui donne un total de 98 bounding boxes prédites par image. 
Pour chacune des cellules de la grille, le réseau ne détecte qu'un seul objet par cellule de la grille, même si deux bounding boxes sont prédites dans chacune des cellules. 

Chaque bounding boxe contient 5 éléments {(x, y, w, h) et un score de confiance de la box qui dit à quel point la box est susceptible de contenir un objet et à quel point la bounding box est précise}. Puisque chacune des cellules prédit deux bounding boxes, elle prédit 5x2 = 10 éléments. De plus, pour chacune des cellules, un score d'appartenance à chacune des 20 classes est prédit, ce qui fait que chacune des cellules de l'image renvoie 20+10 = 30 éléments. Ce qui explique que la taille de la sortie soit 7x7x30. 

La façon dont les bounding boxes sont prédites se base sur la feature map calculée par le réseau de neurones. Chacune des valeurs d'une feature map est calculée en fonction d'un receptive field. Le receptive field d'une cellule est en général plus grand que la taille de la feature map. Ce qui signifie que chacune des cellules contient des informations qui concerne les cellules avoisinantes. En considérant le contenu de la feature map et de celui des bounding boxes générées au centre de chacune des cellules, le réseau va donc pouvoir se concentrer sur chacune des bounding boxes et prédire un décalage des coordonnées du centre ainsi que la différence de hauteur et la différence de largeur. Si le décalage des coordonnées du centres prédites se situent à l'extérieur de la cellule qui a généré les bounding boxes, ce n'est pas cette cellule mais celle contenant le centre de l'objet qui sera retenue pour la prédiction finale. De cette façon il n'y a plus la contrainte d'utiliser une fenêtre glissante pour calculer des prédictions de bounding boxes qui ralenti la détection.

**Performances du réseau** : Très rapide (45 fps pour YOLO, 150 fps pour Fast YOLO mais la précision de détection n'est pas optimale). SSD est un très grand compétiteur de ce réseau pour la détection en temps réel.

Lien vers l'article : https://arxiv.org/pdf/1506.02640.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau :

http://christopher5106.github.io/object/detectors/2017/08/10/bounding-box-object-detectors-understanding-yolo.html

https://stats.stackexchange.com/questions/309345/how-does-the-yolo-network-create-boundaries-for-object-detection

https://stackoverflow.com/questions/50575301/yolo-object-detection-how-does-the-algorithm-predict-bounding-boxes-larger-than

https://hackernoon.com/understanding-yolo-f5a74bbc7967

https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088

https://stats.stackexchange.com/questions/295766/you-only-look-once-yolo-convolutional-neural-network

https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/

## YOLO v2 ##

**Différences avec YOLO v1** : 
- L'image est découpée en une grille de taille 13x13 au lieu de 7x7 ;
- Chacune des grilles prédit 5 bounding boxes différentes ;
- Le réseau de base est Darknet-19 au lieu de VGG-16 ;

**Architecture globale** : Au lieu d'utiliser VGG-16 pour la base du réseau, on utilise un réseau appelé Darknet-19 qui comporte 19 couches de convolution. 

**Architecture détaillée** : 

| Nom de la couche | Taille du noyau, du filtre ou de la couche | Nombre de filtres de convolution | Stride |
| :------------ | :------------ | :------------ | :------------ |
| Convolution Layer | 3x3 | 32 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolution Layer | 3x3 | 64 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 3x3 | 128 | 1 |
| Convolutional Layer | 1x1 | 64 | 1 |
| Convolutional Layer | 3x3 | 128 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Maxpool Layer | 2x2 | N/A | 2 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Convolutional Layer | 1x1 | 1000 | 1 |
| Fully Connected Layer | 13x13x2048 | N/A | N/A |
| Fully Connected Layer | 1x4096 | N/A | N/A |
| Sortie | 13x13x45 | N/A | N/A |

Le réseau Darknet-19 est tout d'abord entraîné sur des images d'ImageNet de taille 224x224 et ensuite entraîné sur 10 épochs sur des images de taille 448x448. Cela permet au réseau de réajuster ses filtres afin qu'il puisse mieux s'adapter à des images de plus grande résolution en entrée. Cela permet d'augmenter de 4% le mAP.

Pour chacune des bounding boxes, 5 caractéristiques sont prédites (coordonnées centre, largeur, hauteur et contient ou pas un objet) ce qui fait 5x5 = 25 caractéristiques. De plus s'ajoutent à ça la probabilité d'appartenance à chacune des 20 classes ce qui donne 20+25 = 45, ce qui explique que la taille en sortie soit de 13x13x45.

**Performances du réseau** : Meilleure précision de détection que YOLO.

Lien vers l'article : https://pjreddie.com/media/files/papers/YOLO9000.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau :

https://towardsdatascience.com/training-object-detection-yolov2-from-scratch-using-cyclic-learning-rates-

http://ethereon.github.io/netscope/#/gist/d08a41711e48cf111e330827b1279c31

https://medium.com/@akarshzingade/logo-detection-using-yolov2-8cda5a68740e

## YOLO 9000 ##

**Architecture globale** : Même architecture que YOLOv2 sauf que YOLO9000 est entraîné pour détecter 9000 classes différentes.

Lien vers l'article : https://pjreddie.com/media/files/papers/YOLO9000.pdf

## YOLO v3 ##

**Différences avec YOLO v2** : 
- 53 couches de convolution au lieu de 19 ;
- Plus précis que YOLO v2 ;
- Moins rapide ;
- Détection à trois échelles différentes.

**Architecture globale** : 53 couches de convolution et deux couches fully connected. 

**Architecture détaillée** :

| Nom de la couche | Taille du noyau, du filtre ou de la couche | Nombre de filtres de convolution | Stride |
| :------------ | :------------ | :------------ | :------------ |
| Convolution Layer | 3x3 | 32 | 1 |
| Convolution Layer | 3x3 | 64 | 2 |
| Convolution Layer | 1x1 | 32 | 1 |
| Convolution Layer | 3x3 | 64 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolution Layer | 3x3 | 128 | 2 |
| Convolutional Layer | 1x1 | 64 | 1 |
| Convolutional Layer | 3x3 | 128 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 64 | 1 |
| Convolutional Layer | 3x3 | 128 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 3x3 | 256 | 2 |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 128 | 1 |
| Convolutional Layer | 3x3 | 256 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 3x3 | 512 | 2 |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 256 | 1 |
| Convolutional Layer | 3x3 | 512 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 3x3 | 1024 | 2 |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Convolutional Layer | 1x1 | 512 | 1 |
| Convolutional Layer | 3x3 | 1024 | 1 |
| Residual Layer | N/A | N/A | N/A |
| Global Average Pooling Layer | 1x1 | 1000 | 1 |
| Fully Connected Layer | 13x13x2048 | N/A | N/A |
| Fully Connected Layer | 1x4096 | N/A | N/A |
| Sortie | 13x13x45 | N/A | N/A |

Les bounding boxes sont prédites de la même façon que pour YOLO v2. La seule différence est que pour prédire la localisation d'objet à trois échelles différentes, le système extrait les caractéristiques de l'image à ces trois échelles en se basant sur un concept similaire aux feature pyramid Network. 

Pour chaque image, 3 bounding boxes sont prédites à chaque échelle. A chaque échelle, le tenseur en sortie est donc de taille 13x13x{3x(4+1+80)} car il y a 80 classes à prédire. Ensuite, on prend la feature map venant de deux layer précédents et on multiplie son échelle par 2. On prend aussi 

Lien vers l'article : https://pjreddie.com/media/files/papers/YOLOv3.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau :

https://towardsdatascience.com/yolo-v3-object-detection-

https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/

## SSD ##

Lien vers l'article : https://arxiv.org/pdf/1708.02002.pdf

Autres liens utiles pour comprendre le fonctionnement du réseau :

https://arxiv.org/pdf/1612.03144.pdf

## RetinaNet ##

## Detectron ##